<a href="https://colab.research.google.com/github/reparadise/MachinelearningProjectAifel/blob/main/%EC%9E%91%EC%82%AC%EA%B0%80AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import numpy as np
import tensorflow as tf
import os


file_path = "/content/drive/MyDrive/shakespeare (1).txt"

# 파일 열기
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines() # 텍스트를 라인 단위로 끊어서 list 형태로 읽어온다.

print(raw_corpus[:9])


['This is the 100th Etext file presented by Project Gutenberg, and', 'is presented in cooperation with World Library, Inc., from their', 'Library of the Future and Shakespeare CDROMS.  Project Gutenberg', 'often releases Etexts that are NOT placed in the Public Domain!!', '', 'Shakespeare', '', '*This Etext has certain copyright implications you should read!*', '']


In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
corpus = []

# 모든 문장에 전처리 함수 적용
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    corpus.append(preprocess_sentence(sentence))

corpus[:10]

['<start> this is the th etext file presented by project gutenberg , and <end>',
 '<start> is presented in cooperation with world library , inc . , from their <end>',
 '<start> library of the future and shakespeare cdroms . project gutenberg <end>',
 '<start> often releases etexts that are not placed in the public domain ! ! <end>',
 '<start> shakespeare <end>',
 '<start> this etext has certain copyright implications you should read ! <end>',
 '<start> this electronic version of the complete works of william <end>',
 '<start> shakespeare is copyright by world library , inc . , and is <end>',
 '<start> provided by project gutenberg etext of illinois benedictine college <end>',
 '<start> with permission . electronic and machine readable copies may be <end>']

In [6]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   29   17 ...    0    0    0]
 [   2   17 3771 ...    0    0    0]
 [   2  465   10 ...    0    0    0]
 ...
 [   2  498    5 ...    0    0    0]
 [   2  265   14 ...    0    0    0]
 [   2  317   10 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa974755290>


단어 사전의 인덱스(idx) 출력

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   29   17    6  117  427 3383 3771   39  425  456    4    7    3
    0    0    0    0    0    0    0    0    0    0    0    0    0]
[  29   17    6  117  427 3383 3771   39  425  456    4    7    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 27), dtype=tf.int32, name=None), TensorSpec(shape=(256, 27), dtype=tf.int32, name=None))>

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 모델의 데이터 확인
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)
# 모델의 최종 출력 shape는 (256, 20, 7001)
# 256은 batch_size, 20은 squence_length, 7001은 단어의 갯수(Dense Layer 출력 차원 수)

<tf.Tensor: shape=(256, 27, 7001), dtype=float32, numpy=
array([[[-1.57204486e-04, -3.55278811e-04,  4.96712804e-04, ...,
         -6.62425955e-05,  1.90058374e-04, -3.11080439e-05],
        [-3.72758979e-04, -8.08644865e-04,  7.61247415e-04, ...,
         -1.56980706e-04,  6.89714449e-04, -3.91661859e-04],
        [-2.12946630e-04, -8.86179507e-04,  8.75313068e-04, ...,
         -4.17742674e-04,  6.67281100e-04, -2.83382018e-04],
        ...,
        [-1.83429185e-03, -2.03509015e-04,  4.01927764e-03, ...,
          4.25135164e-04,  6.99322030e-04,  2.26664310e-03],
        [-1.87037652e-03, -1.78568152e-04,  4.04427387e-03, ...,
          4.90154256e-04,  7.54226872e-04,  2.23689224e-03],
        [-1.90062530e-03, -1.51533910e-04,  4.06415947e-03, ...,
          5.49929973e-04,  8.01073737e-04,  2.20820005e-03]],

       [[-1.57204486e-04, -3.55278811e-04,  4.96712804e-04, ...,
         -6.62425955e-05,  1.90058374e-04, -3.11080439e-05],
        [ 1.39627555e-05, -3.83083825e-04,  4.

In [12]:
# 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = 
(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
445/445 [==============================] - 64s 135ms/step - loss: 2.3021
Epoch 2/30
445/445 [==============================] - 60s 135ms/step - loss: 1.9490
Epoch 3/30
445/445 [==============================] - 60s 135ms/step - loss: 1.8358
Epoch 4/30
445/445 [==============================] - 60s 135ms/step - loss: 1.7686
Epoch 5/30
445/445 [==============================] - 60s 135ms/step - loss: 1.7198
Epoch 6/30
445/445 [==============================] - 60s 135ms/step - loss: 1.6788
Epoch 7/30
445/445 [==============================] - 60s 135ms/step - loss: 1.6428
Epoch 8/30
445/445 [==============================] - 60s 135ms/step - loss: 1.6091
Epoch 9/30
445/445 [==============================] - 60s 135ms/step - loss: 1.5769
Epoch 10/30
445/445 [==============================] - 60s 135ms/step - loss: 1.5458
Epoch 11/30
445/445 [==============================] - 60s 135ms/step - loss: 1.5151
Epoch 12/30
445/445 [==============================] - 60s 135ms/step - lo

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어를 하나씩 생성
    while True:
        predict = model(test_tensor)  # 입력받은 문장
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 예측한 단어가 새로 생성된 단어 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙이기
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종 생성된 자연어 문장

# 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love thee not , nor my <unk> so <end> '